In [ ]:
!nvidia-smi
import torch
import os
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics import YOLO
from DyFilterAttack.mask_attack.utils.CustomDataset import CustomDataset
# os.chdir(path='E://bmx/DyFilterAttack')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = dict(model="../models/yolov8-gtsrb.pt", data="./data.yaml")
trainer = DetectionTrainer(overrides=args)
trainer.setup_model()
trainer.set_model_attributes()
trainer.model.to(device=device)
trainer.model.train()
detector = YOLO(model="../models/yolov8-gtsrb.pt")
detector.to(device=device, dtype=torch.float32).eval()

In [ ]:
from analyzer.utils import analysis_all_layer_cross_classes
# key_filters = analysis_all_layer_cross_classes(trainer)
key_filters = {
    'backbone_c2f1.cv1': [10, 13, 16],
    'backbone_c2f1.cv2': [27],
    'backbone_c2f1.m.0.cv1': [],
    'backbone_c2f1.m.0.cv2': [],
    'backbone_c2f2.cv1': [7, 18, 21],
    'backbone_c2f2.cv2': [21, 38, 45],
    'backbone_c2f2.m.0.cv1': [6, 9, 12, 28, 29, 31],
    'backbone_c2f2.m.0.cv2': [6, 7, 24],
    'backbone_c2f2.m.1.cv1': [7],
    'backbone_c2f2.m.1.cv2': [],
    'backbone_c2f3.cv1': [12, 20, 21, 22, 39, 58, 61],
    'backbone_c2f3.cv2': [0, 41, 96],
    'backbone_c2f3.m.0.cv1': [28],
    'backbone_c2f3.m.0.cv2.conv': [51],
    'backbone_c2f3.m.1.cv1': [],
    'backbone_c2f3.m.1.cv2': [],
    'backbone_c2f4.cv1': [48],
    'backbone_c2f4.cv2': [91, 122],
    'backbone_c2f4.m.0.cv1': [73, 84],
    'backbone_c2f4.m.0.cv2': [120],
    'neck_c2f1.cv1': [25],
    'neck_c2f1.cv2': [],
    'neck_c2f1.m.0.cv1': [40, 59],
    'neck_c2f1.m.0.cv2': [],
    'neck_c2f2.cv1': [1, 52],
    'neck_c2f2.cv2': [],
    'neck_c2f2.m.0.cv1': [24],
    'neck_c2f2.m.0.cv2': [],
    'neck_c2f3.cv1': [7, 114],
    'neck_c2f3.cv2': [44],
    'neck_c2f3.m.0.cv1': [],
    'neck_c2f3.m.0.cv2': [47, 59],
    'neck_c2f4.cv1': [],
    'neck_c2f4.cv2': [228],
    'neck_c2f4.m.0.cv1': [],
    'neck_c2f4.m.0.cv2': [10, 126]
}

In [ ]:
from mask_attack.utils.attacker import Attacker
from DyFilterAttack.mask_attack.utils.datasets.gtsrb import classes_batch_attack_gtsrb
origin_dataset = CustomDataset(images_dir_path = "../../gtsrb_classes_attacked_test/origin/0/images", 
                              labels_dir_path = "../../gtsrb_classes_attacked_test/origin/0/labels",
                              image_width=640,
                              image_height=640)
attacker = Attacker(trainer=trainer, dataset=origin_dataset, batch_size=16)
classes_batch_attack_gtsrb(trainer, classes_name=['0'], batch_size=12,
                           test_classes_root='../../gtsrb_classes_attacked_test/origin/', output_root='./result',
                           method='filter_attack', epsilon=0.05, lr=0.001, num_iter=200, 
                           lambda1=0.5, lambda2=1.5, lambda3=1.0, 
                           key_filters=key_filters)

0.1 1.0 1.5 这个趋势是对的
loss2 涨几个epoch就不变了，但是此时loss1都还没约束好
loss2 需要优化，比如尺度问题，最好三个loss的尺度差不多，这在提取激活值的时候可以修改
做不到同时约束

激活值是个全局的量，如何优化成掩码

In [ ]:
from analyzer.utils import analysis_all_layer
analysis_all_layer(detector, '../../gtsrb_classes_attacked_test/origin/0/images', './result/filter_attack_0-0500_0-0010_200_0.5_1.5_0/0/images', show=True)